### Anomaly Detection Mini Project
#### Corey Solitaire
`11.09.2020`

<div class="alert alert-block alert-info"><b></b></div>

#### Imports:

In [1]:

import wrangle
import explore
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
import seaborn as sns

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

<div class="alert alert-block alert-info"><b></b></div>

### Import wrangle df:

In [2]:
df = wrangle.read_csv()
df = wrangle.wrangle_df(df)
df.head(1)

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61,2018,1,26,9,Friday,1634276157


In [3]:
df.tail(1)

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-11-02 16:48:47,javascript-i/bom-and-dom/bom,763,62,107.192.148.199,2020,11,2,16,Monday,1807783111


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 719459 entries, 2018-01-26 09:55:03 to 2020-11-02 16:48:47
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   page_viewed  719458 non-null  object
 1   user_id      719459 non-null  object
 2   cohort_id    674619 non-null  object
 3   ip           719459 non-null  object
 4   year         719459 non-null  object
 5   month        719459 non-null  object
 6   day          719459 non-null  object
 7   hour         719459 non-null  object
 8   weekday      719459 non-null  object
 9   int_ip       719459 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 60.4+ MB


In [5]:
df = df.fillna(0)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 719459 entries, 2018-01-26 09:55:03 to 2020-11-02 16:48:47
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   page_viewed  719459 non-null  object 
 1   user_id      719459 non-null  int64  
 2   cohort_id    719459 non-null  float64
 3   ip           719459 non-null  object 
 4   year         719459 non-null  int64  
 5   month        719459 non-null  int64  
 6   day          719459 non-null  int64  
 7   hour         719459 non-null  int64  
 8   weekday      719459 non-null  object 
 9   int_ip       719459 non-null  int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 60.4+ MB


In [7]:
cohort_id = df.cohort_id.unique().tolist()
sorted(cohort_id, reverse = True)

[62.0,
 61.0,
 59.0,
 58.0,
 57.0,
 56.0,
 55.0,
 53.0,
 52.0,
 51.0,
 34.0,
 33.0,
 32.0,
 31.0,
 29.0,
 28.0,
 27.0,
 26.0,
 25.0,
 24.0,
 23.0,
 22.0,
 21.0,
 19.0,
 18.0,
 17.0,
 16.0,
 15.0,
 14.0,
 13.0,
 12.0,
 11.0,
 9.0,
 8.0,
 7.0,
 6.0,
 5.0,
 4.0,
 2.0,
 1.0,
 0.0]

`Explore Missing user_id Further`

<div class="alert alert-block alert-info"><b></b></div>

# Questions to Answer:
****
  1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?


  2. Is there a cohort that referred to a lesson significantly more that other cohorts seemed to gloss over? 


  3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students? 


  4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses? Any odd user-agents? 


  5. At some point in the last year, ability for students and alumni to cross-access curriculum (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before? 


  6. What topics are grads continuing to reference after graduation and into their jobs (for each program)? 


  7. Which lessons are least accessed? 


  8. Anything else I should be aware of? 

<div class="alert alert-block alert-info"><b></b></div>

## First Step is to ID Cohorts:

explore.cohort_id(df, year, month, day, hour)
***

In [8]:
# 21,Sequoia,2017-09-27,2018-02-15,2
# Cohort ID #21 I think

In [9]:
# 22,Teddy,2018-01-08,2018-05-17,2 
# test = explore.cohort_id(df,2018,1,26,10)
# test.head()

#### Cohort Teddy

In [10]:
teddy = df[df.cohort_id == 22]
len(teddy.user_id.unique())

35

In [11]:
teddy.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2018,1,26,9,Friday,1634276157
2018-01-26 09:56:41,javascript-i/loops,2,22.0,97.105.19.61,2018,1,26,9,Friday,1634276157
2018-01-26 09:56:46,javascript-i/conditionals,3,22.0,97.105.19.61,2018,1,26,9,Friday,1634276157
2018-01-26 09:56:48,javascript-i/functions,3,22.0,97.105.19.61,2018,1,26,9,Friday,1634276157
2018-01-26 09:56:59,javascript-i/loops,2,22.0,97.105.19.61,2018,1,26,9,Friday,1634276157


In [12]:
# 23,Ulysses,2018-03-05,2018-07-19,2
# test = explore.cohort_id(df,2018,3,5,10)
# test.head()

#### Cohort Ulysses

In [13]:
ulysses = df[df.cohort_id == 23]
len(ulysses.user_id.unique())

33

In [14]:
ulysses.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-03-05 14:22:48,html-css/css-i,25,23.0,97.105.19.61,2018,3,5,14,Monday,1634276157
2018-03-05 14:22:58,html-css/forms,25,23.0,97.105.19.61,2018,3,5,14,Monday,1634276157
2018-03-05 14:23:06,html-css/css-i,25,23.0,97.105.19.61,2018,3,5,14,Monday,1634276157
2018-03-05 14:23:08,html-css/css-i/introduction,25,23.0,97.105.19.61,2018,3,5,14,Monday,1634276157
2018-03-05 14:23:47,html-css,25,23.0,97.105.19.61,2018,3,5,14,Monday,1634276157


In [15]:
# 24,Voyageurs,2018-05-29,2018-10-11,2
# test = explore.cohort_id(df,2018,5,29,10)
# test.head()

#### Cohort Voyagers

In [16]:
voyagers = df[df.cohort_id == 24]
len(voyagers.user_id.unique())

34

In [17]:
voyagers.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-05-29 14:42:38,/,182,24.0,97.105.19.61,2018,5,29,14,Tuesday,1634276157
2018-05-29 14:42:39,/,183,24.0,97.105.19.61,2018,5,29,14,Tuesday,1634276157
2018-05-29 14:42:42,/,184,24.0,97.105.19.61,2018,5,29,14,Tuesday,1634276157
2018-05-29 14:42:42,/,185,24.0,97.105.19.61,2018,5,29,14,Tuesday,1634276157
2018-05-29 14:42:42,/,186,24.0,97.105.19.61,2018,5,29,14,Tuesday,1634276157


In [18]:
# 25,Wrangell,2018-07-23,2018-11-29,2
# test = explore.cohort_id(df,2018,7,23,10)
# test.head()

#### Cohort Wranglell

In [19]:
wranglell = df[df.cohort_id == 25]
len(wranglell.user_id.unique())

26

In [20]:
wranglell.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-07-23 13:51:26,registerUser,218,25.0,97.105.19.58,2018,7,23,13,Monday,1634276154
2018-07-23 13:51:39,registerUser,218,25.0,97.105.19.58,2018,7,23,13,Monday,1634276154
2018-07-23 15:15:19,/,219,25.0,97.105.19.58,2018,7,23,15,Monday,1634276154
2018-07-23 15:15:20,/,220,25.0,97.105.19.58,2018,7,23,15,Monday,1634276154
2018-07-23 15:15:21,/,221,25.0,97.105.19.58,2018,7,23,15,Monday,1634276154


In [21]:
# 26,Xanadu,2018-09-17,2019-02-08,2
# test = explore.cohort_id(df,2018,9,17,10)
# test.head()

#### Cohort Xanadu

In [22]:
xanadu = df[df.cohort_id == 26]
len(xanadu.user_id.unique())

20

In [23]:
xanadu.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-09-17 14:12:28,/,259,26.0,97.105.19.58,2018,9,17,14,Monday,1634276154
2018-09-17 14:13:38,/,260,26.0,97.105.19.58,2018,9,17,14,Monday,1634276154
2018-09-17 14:13:42,html-css,260,26.0,97.105.19.58,2018,9,17,14,Monday,1634276154
2018-09-17 14:13:58,/,261,26.0,97.105.19.58,2018,9,17,14,Monday,1634276154
2018-09-17 14:14:21,/,262,26.0,97.105.19.58,2018,9,17,14,Monday,1634276154


In [24]:
# 27,Yosemite,2018-11-05,2019-04-03,2
# test = explore.cohort_id(df,2018,11,5,10)
# test.head(20)

#### Cohort Yosemite

In [25]:
yosemite = df[df.cohort_id == 27]
len(yosemite.user_id.unique())

29

In [26]:
yosemite.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-11-05 14:03:58,/,287,27.0,97.105.19.58,2018,11,5,14,Monday,1634276154
2018-11-05 14:04:58,/,288,27.0,97.105.19.58,2018,11,5,14,Monday,1634276154
2018-11-05 14:05:18,/,289,27.0,97.105.19.58,2018,11,5,14,Monday,1634276154
2018-11-05 14:06:41,/,290,27.0,97.105.19.58,2018,11,5,14,Monday,1634276154
2018-11-05 14:06:51,/,291,27.0,97.105.19.58,2018,11,5,14,Monday,1634276154


****

In [27]:
# 28,Staff,2014-02-04,2014-02-04,2

`Staff are listed under the webdev cohorts (2)`

****

In [28]:
# 29,Zion,2019-01-22,2019-06-04,2
test = explore.cohort_id(df,2019,1,22,9)
test.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-01-22 09:02:00,spring,272,26.0,97.105.19.58,2019,1,22,9,Tuesday,1634276154
2019-01-22 09:02:05,spring/fundamentals/form-model-binding,272,26.0,97.105.19.58,2019,1,22,9,Tuesday,1634276154
2019-01-22 09:02:23,java-ii/annotations,295,27.0,97.105.19.58,2019,1,22,9,Tuesday,1634276154
2019-01-22 09:02:37,java-ii/annotations,289,27.0,97.105.19.58,2019,1,22,9,Tuesday,1634276154
2019-01-22 09:02:38,java-ii/collections,292,27.0,97.105.19.58,2019,1,22,9,Tuesday,1634276154


#### Cohort Zion

In [29]:
zion = df[df.cohort_id == 29]
len(zion.user_id.unique())

25

In [30]:
zion.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-01-22 15:09:54,/,323,29.0,97.105.19.58,2019,1,22,15,Tuesday,1634276154
2019-01-22 15:10:01,toc,323,29.0,97.105.19.58,2019,1,22,15,Tuesday,1634276154
2019-01-22 15:11:47,/,324,29.0,97.105.19.58,2019,1,22,15,Tuesday,1634276154
2019-01-22 15:11:53,toc,323,29.0,97.105.19.58,2019,1,22,15,Tuesday,1634276154
2019-01-22 15:11:55,/,325,29.0,97.105.19.58,2019,1,22,15,Tuesday,1634276154


In [ ]:
# zion_id = test.user_id.unique().tolist()
# sorted(zion_id, reverse = True)

In [32]:
# 30,Ada,2019-02-04,2019-06-16,3
# test = explore.cohort_id(df,2019,2,4,9)
# test.tail(5)

#### Cohort Ada

In [58]:
ada = df[df.cohort_id == 0]
len(ada.user_id.unique())

75

In [59]:
ada.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip,ada
datetime,,,,,,,,,,,
2018-01-26 16:46:16,/,48,0.0,97.105.19.61,2018,1,26,16,Friday,1634276157,True
2018-01-26 16:46:24,spring/extra-features/form-validation,48,0.0,97.105.19.61,2018,1,26,16,Friday,1634276157,True
2018-01-26 17:54:24,/,48,0.0,97.105.19.61,2018,1,26,17,Friday,1634276157,True
2018-01-26 18:32:03,/,48,0.0,97.105.19.61,2018,1,26,18,Friday,1634276157,True
2018-01-26 18:32:17,mysql/relationships/joins,48,0.0,97.105.19.61,2018,1,26,18,Friday,1634276157,True


In [72]:
# To find ada must find not ada first
ada = ada[ada.user_id != 48]
ada = ada[ada.user_id != 54]
ada = ada[ada.user_id != 58]
ada = ada[ada.user_id != 59]
ada = ada[ada.user_id != 61]
ada = ada[ada.user_id != 62]
ada = ada[ada.user_id != 63]
ada = ada[ada.user_id != 73]
ada = ada[ada.user_id != 74]
ada = ada[ada.user_id != 78]
ada = ada[ada.user_id != 79]
ada = ada[ada.user_id != 86]
ada = ada[ada.user_id != 88]
ada = ada[ada.user_id != 89]
ada = ada[ada.user_id != 100]
ada.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip,ada
datetime,,,,,,,,,,,
2018-02-16 16:38:04,/,103,0.0,64.157.241.246,2018,2,16,16,Friday,1084092918,True
2018-02-28 18:12:27,content/javascript/loops.html,111,0.0,70.117.6.133,2018,2,28,18,Wednesday,1182074501,True
2018-02-28 18:12:32,/,111,0.0,70.117.6.133,2018,2,28,18,Wednesday,1182074501,True
2018-02-28 18:12:37,javascript-i,111,0.0,70.117.6.133,2018,2,28,18,Wednesday,1182074501,True
2018-02-28 19:30:32,javascript-i/loops,111,0.0,70.117.6.133,2018,2,28,19,Wednesday,1182074501,True


In [84]:
ada.head(1200)

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip,ada
datetime,,,,,,,,,,,
2018-02-16 16:38:04,/,103,0.0,64.157.241.246,2018,2,16,16,Friday,1084092918,True
2018-02-28 18:12:27,content/javascript/loops.html,111,0.0,70.117.6.133,2018,2,28,18,Wednesday,1182074501,True
2018-02-28 18:12:32,/,111,0.0,70.117.6.133,2018,2,28,18,Wednesday,1182074501,True
2018-02-28 18:12:37,javascript-i,111,0.0,70.117.6.133,2018,2,28,18,Wednesday,1182074501,True
2018-02-28 19:30:32,javascript-i/loops,111,0.0,70.117.6.133,2018,2,28,19,Wednesday,1182074501,True
...,...,...,...,...,...,...,...,...,...,...,...
2019-02-07 13:47:30,2.00.01_Intro_Excel,360,0.0,97.105.19.58,2019,2,7,13,Thursday,1634276154,True
2019-02-07 13:50:26,2.00.00_Excel_Prob_Stats,360,0.0,97.105.19.58,2019,2,7,13,Thursday,1634276154,True
2019-02-07 13:50:34,2.00.02_Navigating_Excel,360,0.0,97.105.19.58,2019,2,7,13,Thursday,1634276154,True


In [35]:
# 31,Andromeda,2019-03-18,2019-07-30,2
test = explore.cohort_id(df,2019,3,18,9)
test.tail()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-03-18 09:54:04,Intro_to_Regression,363,0.0,97.105.19.58,2019,3,18,9,Monday,1634276154
2019-03-18 09:54:04,ordinary_least_squares.jpeg,363,0.0,97.105.19.58,2019,3,18,9,Monday,1634276154
2019-03-18 09:54:05,Univariate_Regression_Excel,248,28.0,97.105.19.58,2019,3,18,9,Monday,1634276154
2019-03-18 09:54:11,Univariate_Regression_Excel,248,28.0,97.105.19.58,2019,3,18,9,Monday,1634276154
2019-03-18 09:56:41,Univariate_Regression_Excel,355,0.0,97.105.19.58,2019,3,18,9,Monday,1634276154


In [36]:
test.user_id.unique()

array([362, 357, 248, 363, 372, 353, 351, 355, 361, 359, 349, 364, 354,
       369, 367, 347, 344, 337, 368, 366, 330, 352, 328, 266])

In [37]:
ada_id = test.user_id.unique().tolist()
sorted(ada_id, reverse = True)

[372,
 369,
 368,
 367,
 366,
 364,
 363,
 362,
 361,
 359,
 357,
 355,
 354,
 353,
 352,
 351,
 349,
 347,
 344,
 337,
 330,
 328,
 266,
 248]

In [38]:
# Next Hour
test = explore.cohort_id(df,2019,3,18,10)
test.tail()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-03-18 10:56:36,3.0-mysql-overview,355,0.0,97.105.19.58,2019,3,18,10,Monday,1634276154
2019-03-18 10:56:41,3.8.2-joins,355,0.0,97.105.19.58,2019,3,18,10,Monday,1634276154
2019-03-18 10:56:42,3.8.3-subqueries,355,0.0,97.105.19.58,2019,3,18,10,Monday,1634276154
2019-03-18 10:56:43,3.9-temporary-tables,355,0.0,97.105.19.58,2019,3,18,10,Monday,1634276154
2019-03-18 10:56:49,appendix,11,28.0,97.105.19.58,2019,3,18,10,Monday,1634276154


In [39]:
ada_id = test.user_id.unique().tolist()
sorted(ada_id, reverse = True)

[372,
 369,
 368,
 367,
 366,
 363,
 361,
 359,
 357,
 355,
 354,
 353,
 352,
 351,
 346,
 344,
 337,
 336,
 330,
 314,
 248,
 11]

#### Cohort: Andromeda

In [42]:
andromeda = df[df.cohort_id == 31]

In [43]:
# 32,Betelgeuse,2019-05-28,2019-10-08,2

In [44]:
# 33,Ceres,2019-07-15,2019-12-11,2

In [45]:
# 34,Bayes,2019-08-19,2020-01-30,3

In [46]:
# 51,Deimos,2019-09-16,2020-02-27,2

In [47]:
# 52,Europa,2019-11-04,2020-04-17,2

In [48]:
# 53,Fortuna,2020-01-13,2020-06-23,2

In [49]:
# 55,Curie,2020-02-03,2020-07-07,3

In [50]:
# 56,Apex,2020-02-24,2020-07-29,2

In [51]:
# 57,Ganymede,2020-03-23,2020-08-20,2

In [52]:
# 58,Hyperion,2020-05-26,2020-11-10,2

In [53]:
# 59,Darden,2020-07-13,2021-01-12,3

In [54]:
# 60,Io,2020-07-20,2021-01-21,2

In [55]:
# 61,Bash,2020-07-20,2021-01-21,2

In [56]:
# 62,Jupiter,2020-09-21,2021-03-30,2

In [57]:
# 132,Kalypso,2020-11-02,2021-05-04,2